In [31]:
"""
This notebook is used to preprocess the different juridical codes of the penal law.
The codes are downloaded from the website of the French government (https://www.legifrance.gouv.fr/).

Currently we only have three codes:
- Code pénal
- Code civique
- Code du travail

The codes are downloaded in the folder "data/codepenal" and are named "codepenal.txt", "codecivil.txt" and "codetravail.txt".

In this notebook we preprocess the codes to extract the articles and the paragraphs.

We have two ideas for the preprocessing:
- Extract the full text of the articles and the paragraphs
OR
- Extract each article separately 
"""

# Imports
import PyPDF2
import numpy as np
import pandas as pd

### 1. Extract raw text from the dataset

In [2]:
files_paths = ["../data/Codecivil.pdf", "../data/Codepenal.pdf", "../data/Codedutravail.pdf"]

In [5]:

def extract_text_from_pdf(file_path):
    """
    Extract the text from a PDF file and save it to a text file.
    """
    # Open the PDF file
    pdf_file = open(file_path, 'rb')

    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    # Get the number of pages in the PDF document
    num_pages = len(pdf_reader.pages)

    # Iterate through all the pages and extract the text
    text = ''
    for page in range(num_pages):
        page_obj = pdf_reader.pages[page]
        text += page_obj.extract_text()

    return text




In [7]:

# Extract the text from the PDF files
text_civil = extract_text_from_pdf("../data/Codecivil.pdf")

In [9]:
text_civil

'Code civil\nDernière modification: 2023-02-06\nEdition : 2023-04-09\n2881 articles avec 1296 liens\n2078 références externes\nCe code ne contient que du droit positif français,\nles articles et éléments abrogés ne sont pas inclus.\nIl est recalculé au fur et à mesure des mises à jour.\nPensez à actualiser votre copie régulièrement à partir de codes.droit.org .\nCes codes ont pour objectif de démontrer l’utilité de l’ouverture des données publiques\njuridiques tant législatives que jurisprudentielles. Il s’y ajoute une promotion du mouvement\nOpen Science Juridique avec une incitation au dépôt du texte intégral en accès ouvert\ndes articles de doctrine venant du monde professionnel (Grande Bibliothèque du Droit) et\nuniversitaire (HAL-CNRS).\nTraitements effectués à partir des données issues des APIs Legifrance et Judilibre. droit.org\nremercie les acteurs du Web qui autorisent des liens vers leur production : Dictionnaire du\nDroit Privé  (réalisé par MM. Serge Braudo et Alexis Bauman

In [10]:
# on décompose le texte en sous text en fonction des numéros d'articles
# On a les articles qui sont de la forme :
# NUMERO_ARTICLE
# Legif.   
# Plan
# etc

# on récupère les articles
def split_text(text):
    """
    Split the text into articles.
    """
    articles = text.split("Legif.")
    return articles

articles_civil = split_text(text_civil)




In [16]:
# we check the lenght of the articles
article_len = []
for article in articles_civil:
    print(len(article))
    article_len.append(len(article))

    

123030
1328
606
940
557
400
849
401
799
460
180
1391
868
954
386
905
960
740
376
342
390
673
784
394
396
763
528
817
435
2640
3197
835
278
1033
459
773
458
1097
371
228
410
313
654
365
906
359
819
583
790
512
585
298
234
1216
967
430
437
564
305
852
608
219
2153
344
1161
402
863
865
555
574
514
1975
1532
1016
960
883
1734
769
339
260
392
762
949
725
481
468
524
1481
371
350
1423
1047
1572
779
1433
1331
474
645
225
1055
550
440
411
286
437
819
827
356
889
961
433
377
620
549
973
1768
846
1014
318
1096
1344
849
428
400
555
688
734
1502
485
325
346
899
413
629
617
460
1482
862
539
331
292
990
696
585
431
451
914
573
636
675
399
1776
980
232
257
254
316
471
341
1104
1201
1500
568
1454
289
241
430
707
892
1253
480
2949
517
2038
1926
1149
582
550
275
750
1810
983
913
792
893
677
1129
1091
2943
565
270
397
439
299
388
1200
783
880
444
342
2117
1918
698
1112
1328
1357
655
459
407
385
503
768
1000
858
928
848
891
1687
347
476
2309
260
944
612
998
474
608
671
1012
1024
836
280
484
1251
1448
1424

In [26]:
# we take the 97% quantile to remove the articles that are too long
limit_size = np.quantile(article_len, 0.97)

# we remove the articles that are too long
articles_civil = [article for article in articles_civil if len(article) < limit_size]

# we have now 2795 articles
len(articles_civil)



2795

In [32]:
df_law = pd.DataFrame(articles_civil, columns=["text"]) 



In [46]:
# we preprocess the text to remove the \n at the end and also we retrieve the last line of the string which is the article number
def preprocess_text(text):
    """
    Preprocess the text to retrieve the last line of the string which is the article number.
    """
    # we retrieve the last line of the string which is the article number
    last_line = text.split("\n")[-2]

    # we take the first word of the line that should be the article number
    article_number = last_line.split(" ")[0]

    return article_number

# we apply the function to the dataframe
df_law["article_number"] = df_law["text"].apply(preprocess_text)

# now we shift the article number to the next row
df_law["article_number"] = df_law["article_number"].shift(1)


In [47]:
# we save the dataframe in parquet format
df_law.to_parquet("../data/df_law_civique.parquet")

In [49]:
# check nan values in the dataframe
df_law.isna().sum()

text              0
article_number    1
dtype: int64

In [62]:
# check value of article number 1730, 1755, 1631
df_law[df_law["article_number"] == "652"]["text"].values[0]

"   \n Plan   \n Jp.C.Cass.   \n Jp.Appel   \n Jp.Admin.   \n Juricaf\nPartie de ces obligations est réglée par les lois sur la police rurale ;\nLes autres sont relatives au mur et au fossé mitoyens, au cas où il y a lieu à contre-mur, aux vues sur la propriété\ndu voisin, à l'égout des toits, au droit de passage.\nSection 1 : Du mur et du fossé mitoyens\nservice-public.fr\n> Mur mitoyen : Code civil : articles 653 à 673\n> Maison : travaux extérieurs : Code civil : articles 653 à 673\n> Plantations (haies, arbres, arbustes...) : Code civil : articles 653 à 673\n653    \n "